In [ ]:
%reload_ext pytriqs.magic

In [ ]:
from pytriqs.utility import mpi

In [ ]:
%%triqs --only qmc
#include <triqs/mc_tools/mc_generic.hpp>
#include <triqs/utility/callbacks.hpp>

// --------------- configuration : a spin +1, -1 ---------------

struct configuration {
 int spin = -1;
};

// --------------- a move: flip the spin ---------------
struct flip {
 configuration* config;
 double beta_h;

 flip(configuration* config_, double beta, double h) : config(config_), beta_h(beta * h) {}

 double attempt() { return std::exp(-2 * config->spin * beta_h); }

 double accept() {
  config->spin *= -1;
  return 1.0;
 }

 void reject() {}
};

//  ----------------- a measurement: the magnetization ------------
class compute_m {
 configuration const * config;
 double& avg_magn;
 double Z = 0, M = 0;
 public:   

 compute_m(configuration* config_, double& avg_magn) : config(config_), avg_magn(avg_magn) {}

 void accumulate(double sign) {
  Z += sign;
  M += sign * config->spin;
 }

 void collect_results(triqs::mpi::communicator c) {
  avg_magn = M/Z;
 }
};

//  ----------------- main ------------

double qmc(double beta, double field) { 
  
 triqs::mpi::communicator world;
    
 // #parameters of the Monte Carlo
 int n_cycles = 5000000;
 int length_cycle = 10;
 int n_warmup_cycles = 10000;
 std::string random_name = "";
 int random_seed = 374982 + world.rank() * 273894;
 int verbosity = (world.rank() == 0 ? 2 : 0);

 // #Generic Monte Carlo
 triqs::mc_tools::mc_generic<double> SpinMC(random_name, random_seed, 1.0, verbosity);

 configuration config;
 double mag;

 // #add moves and measures
 SpinMC.add_move(flip(&config, beta, field), "flip move");
 SpinMC.add_measure(compute_m(&config, mag), "magnetization measure");

 // #Run and collect results
 SpinMC.warmup_and_accumulate(n_warmup_cycles, n_cycles, length_cycle, triqs::utility::clock_callback(600));
 SpinMC.collect_results(world);
 //std::cout << "Finished calculation for field = " << field << "." << std::endl;
 return mag;
}


In [ ]:
import numpy as np
X = np.arange(0.1,2,0.2)
r = [qmc(2, h) for h in X]

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
r_theo = [(1- np.exp(-2 * 2*h))/ (1+ np.exp(-2 * 2*h)) for h in X]
plt.plot(X, r, '-o', label='Calculated')
plt.plot(X, r_theo, label='Theoretical')
plt.xlim(0,2)
plt.ylim(0,1)
plt.legend()